# Regressão Logística I
## Tarefa II

Vamos trabalhar com a mesma base do exercício anterior, mas vamos aprofundar um pouco mais a nossa regressão.

In [115]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import roc_curve, auc, confusion_matrix

import statsmodels.formula.api as smf

In [89]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data'

df = pd.read_csv(url,
                 names=['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg',
                        'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num'])
df['flag_doente'] = (df['num']!=0).astype('int64')
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num,flag_doente
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2,1
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0,0


In [90]:
df2 = pd.DataFrame([{"age": 31, "sex": 1, "cp": 1, "trestbps": 145, "chol": 1, "fbs": 1, "restecg": 2, "thalach": 150, "exang": 1, "oldpeak": 2.5, "slope": 3, "ca": 0, "thal": 6, "num": 1, "flag_doente": 1}])

In [91]:
df = pd.concat([df, df2])

A descrição das variáveis está recortada abaixo:
- age: idade do paciente em anos
- sex: sexo (1 = male; 0 = female)  
- cp: tipo de dor no peito
  - 1: angina típica
  - 2: angina atípica
  - 3: dor não-angina
  - 4: assintomático
- trestbps: pressão sanguínea em repouso (em mm Hg na admissão ao hospital
- chol: colesterol sérico em mg/dl
- fbs: (açúcar no sangue em jejum > 120 mg/dl) (1 = True; 0 = False)
- restecg: resultados eletrocardiográficos em repouso
  - 0: normal
  - 1: tendo anormalidade da onda ST-T (Inversões de onda T e / ou ST com elevação ou depressão de > 0.05 mV)
  - 2: mostrando hipertrofia ventricular esquerda provável ou definitiva pelos critérios de Estes
- thalach: frequência cardíaca máxima alcançada
- exang: angina induzida por exercício(1 = sim; 0 = não)
- oldpeak = Depressão de ST induzida por exercício em relação ao repouso
- slope: Depressão de ST induzida por exercício em relação ao repouso
  - 1: inclinação ascendente
  - 2: estável
  - 3: inclinação descendente
- ca: número de vasos principais (0-3) coloridos por fluorosopia
- thal: 3 = normal; 6 = defeito corrigido; 7 = defeito reversível
- num: diagnóstico de doença cardíaga (status de doença angiográfica)

In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 304 entries, 0 to 0
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          304 non-null    float64
 1   sex          304 non-null    float64
 2   cp           304 non-null    float64
 3   trestbps     304 non-null    float64
 4   chol         304 non-null    float64
 5   fbs          304 non-null    float64
 6   restecg      304 non-null    float64
 7   thalach      304 non-null    float64
 8   exang        304 non-null    float64
 9   oldpeak      304 non-null    float64
 10  slope        304 non-null    float64
 11  ca           304 non-null    object 
 12  thal         304 non-null    object 
 13  num          304 non-null    int64  
 14  flag_doente  304 non-null    int64  
dtypes: float64(11), int64(2), object(2)
memory usage: 38.0+ KB


1. Considere o script que monta a análise bivariada que você fez na tarefa anterior. Transforme esse script em uma função, que deve:
- Ter como parâmetros de entrada:
    - Um *dataframe* contendo os dados a serem avaliados
    - Um *string* contendo o nome da variável resposta
    - Um *string* contendo o nome da variável explicativa
- E deve retornar um *dataframe* com os dados da bivariada.
**Monte** a mesma bivariada pelo menos três variáveis qualitativas do *data-frame*. Qual delas parece discriminar mais o risco?

In [93]:
def calcular_metricas(dataframe: pd.DataFrame) -> pd.DataFrame:
  biv = dataframe.copy()
  biv["media"] = biv["total"] / biv[f"sim"]

  biv["odds"] = (biv[f"sim"] / biv["total"]) / (biv[f"não"] / biv["total"])

  biv["odds ratio"] = biv["odds"] / biv["total"]

  biv["logito"] = np.log(biv["odds"])
  biv["woe"] = np.log(biv["odds ratio"])

  return biv

In [94]:
def analise_bivariada(dataframe: pd.DataFrame, response: str, explain: str) -> pd.DataFrame:
  biv = dataframe.copy().groupby([response, explain]).size().reset_index(name='total')

  biv['não'] = biv.apply(lambda row: row['total'] if row[explain] == 0 else 0, axis=1)
  biv['sim'] = biv.apply(lambda row: row['total'] if row[explain] == 1 else 0, axis=1)

  biv = biv.groupby(response).sum().reset_index()

  biv = calcular_metricas(biv)
  biv.drop(columns=[explain], inplace=True)

  return biv

In [95]:
biv_doentes = analise_bivariada(df, "sex", "flag_doente")
biv_fbs = analise_bivariada(df, "sex", "fbs")
biv_exang = analise_bivariada(df, "sex", "exang")

In [96]:
biv_doentes

,sex,total,não,sim,media,odds,odds ratio,logito,woe
0,0.0,97,72.0,25.0,3.88,0.347222,0.003580,-1.057790,-5.632501
1,1.0,207,92.0,115.0,1.80,1.250000,0.006039,0.223144,-5.109575


2. Monte uma função semelhante para categorizar variáveis quantitativas contínuas (com muitas categorias) como ```age```.  
    Além dos mesmos parâmetros da função anterior, defina mais um parâmetro como número de categorias que você deseja quebrar. Defina um valor '*default*' de 5 grupos para este parâmetro.  

In [97]:
def calcular_limites(min, max, groups):
    amplitude = (max - min) / groups
    limites = [min + i * amplitude for i in range(groups + 1)]
    return limites

In [98]:
calcular_limites(df["age"].min(), df["age"].max(), 5)

[29.0, 38.6, 48.2, 57.8, 67.4, 77.0]

In [99]:
def analise_bivariada_por_categoria(dataframe: pd.DataFrame, response: str, explain: str, groups: int):
  biv = dataframe.copy().groupby([response, explain]).size().reset_index(name='total')
  bins = calcular_limites(biv[response].min(), biv[response].max(), groups)
  labels = [f"Group {i}" for i in range(0, groups)]

  biv["group"] = pd.cut(biv[response], bins=bins, labels=labels, right=False)

  biv['não'] = biv.apply(lambda row: row['total'] if row[explain] == 0 else 0, axis=1)
  biv['sim'] = biv.apply(lambda row: row['total'] if row[explain] == 1 else 0, axis=1)

  biv = biv.groupby("group").sum().reset_index()

  biv = calcular_metricas(biv)
  biv.drop(columns=[explain, response], inplace=True)

  return biv

In [100]:
biv = analise_bivariada_por_categoria(df, "age", "fbs", 5)

In [101]:
biv

,group,total,não,sim,media,odds,odds ratio,logito,woe
0,Group 0,12,11,1,12.000000,0.090909,0.007576,-2.397895,-4.882802
1,Group 1,71,65,6,11.833333,0.092308,0.001300,-2.382628,-6.645308
2,Group 2,97,80,17,5.705882,0.212500,0.002191,-1.548813,-6.123524
3,Group 3,107,89,18,5.944444,0.202247,0.001890,-1.598265,-6.271093
4,Group 4,16,12,4,4.000000,0.333333,0.020833,-1.098612,-3.871201


3. Construa um modelo de regressão logística com as variáveis qualitativas: ```sex + cp +  trestbps``` e com a variável quantitativa ```age```.

**Interprete os parâmetros.**

In [102]:
model = smf.logit(formula='flag_doente ~ sex + cp + trestbps + age', data=df)
result = model.fit()

Optimization terminated successfully.
         Current function value: 0.518756
         Iterations 6


In [103]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            flag_doente   No. Observations:                  304
Model:                          Logit   Df Residuals:                      299
Method:                           MLE   Df Model:                            4
Date:                Mon, 22 Apr 2024   Pseudo R-squ.:                  0.2482
Time:                        16:56:06   Log-Likelihood:                -157.70
converged:                       True   LL-Null:                       -209.77
Covariance Type:            nonrobust   LLR p-value:                 1.296e-21
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -10.6464      1.565     -6.803      0.000     -13.714      -7.579
sex            1.7815      0.326      5.458      0.000       1.142       2.421
cp             1.0811      0.164      6.588      0.000       0.759       1.403
trestbps       0.0225      0.008      2.746      0.006       0.006       0.038
age            0.0511      0.017      3.019      0.003       0.018       0.084
==============================================================================
"""

4. Avalie o seu modelo quanto a **calibragem**:
- Calcule a probabilidade de evento predita segundo o seu modelo
- Categorize essa probabilidade em G=5 grupos
- Calcule a probabilidade de evento predita média por grupo
- Calcule a taxa de eventos (média da variável indicadora de eventos) por grupo
- Compare graficamente o valor eperado versus observado para a taxa de maus por grupo

In [104]:
df['prob_predita'] = result.predict(df)

In [106]:
df['grupo'] = pd.qcut(df['prob_predita'], q=5, labels=False)

In [110]:
prob_media_por_grupo = df.groupby('grupo')['prob_predita'].mean()

In [111]:
taxa_eventos_por_grupo = df.groupby('grupo')['flag_doente'].mean()

In [113]:
fig = px.line(
    x=prob_media_por_grupo,
    y=taxa_eventos_por_grupo
)

In [114]:
fig.show()

5. Avalie o seu modelo quanto a discriminação calculando acurácia, GINI e KS.

In [122]:
# Calculando a acurácia
y_pred = np.where(df['prob_predita'] >= 0.5, 1, 0)
accuracy = accuracy_score(df['flag_doente'], y_pred)
accuracy

0.7796052631578947

In [126]:
# Calculando o Gini
fpr, tpr, _ = roc_curve(df['flag_doente'], y_pred)
roc_auc = auc(fpr, tpr)
gini_index = 2 * roc_auc - 1
gini_index

0.5580139372822299

In [129]:
# Estatística KS
ks_statistic = np.max(np.abs(tpr - fpr))
ks_statistic

0.55801393728223

6. tente melhorar o modelo obtido, por exemplo inserindo ou removendo variáveis.  
    Avalie as características do seu modelo (calibragem e acurácia).

In [131]:
df.head(1)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num,flag_doente,prob_predita,grupo
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0,0,0.213127,1


In [132]:
model = smf.logit(formula='flag_doente ~ sex + cp + trestbps + age + chol + fbs + thalach + oldpeak + slope', data=df)
result = model.fit()

Optimization terminated successfully.
         Current function value: 0.436058
         Iterations 7


In [134]:
df['prob_predita'] = result.predict(df)

In [135]:
y_pred = np.where(df['prob_predita'] >= 0.5, 1, 0)
accuracy = accuracy_score(df['flag_doente'], y_pred)
accuracy

0.7894736842105263